# 前置作業
記得要在TERMINAL下載套件 <br> !pip install -q pandas openpyxl requests python-dotenv

# 1.載入套件

In [41]:
import os
import time
import requests
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv
import re

# 2.載入環境變數
- 記得建立一個 .env 檔，裡面存放 API Key (安全起見，避免金鑰外洩，免費的 heygen api，heygen 只有給予10點，很寶貴!!!)

In [51]:
load_dotenv()
HEYGEN_API_KEY = os.getenv("HEYGEN_API_KEY")

# 3.API呼叫端點之設定

In [52]:
API_HOST = "https://api.heygen.com"
GENERATE_URL_V2 = f"{API_HOST}/v2/video/generate"
UPLOAD_URL_V1 = "https://upload.heygen.com/v1/asset"

# 4.Talking Photo ID & voice (方便替換便替換)
- en:代表英文 voice
- zh:代表中文 voice (方便後面定義之選擇)

In [53]:
TALKING_PHOTO_ID = "d9ce6da9c3864166836359ed9460a9b4"
VOICE_ID_EN = "cef3bc4e0a84424cafcde6f2cf466c97"
VOICE_ID_ZH = "4158cf2ef85d4ccc856aacb1c47dbb0c"

# 5.檔案路徑之設定


In [54]:
ROOT = Path(__file__).parent if "__file__" in globals() else Path(os.getcwd())
PNG_DIR = ROOT / "outputs" / "slides_png"
SCRIPT_PATH = ROOT / "scripts" / "script.xlsx"

# 6.定義讀入腳本之功能
- 6.1.用 Panadas 開 excel
- 6.2.將欄位都轉成小寫
- 6.3.文字轉成字串
- 6.4.接著回傳一個 dict

In [55]:
def load_scripts(script_path: Path):
    try:
        df = pd.read_excel(script_path)
        df.columns = [c.lower() for c in df.columns]
        df["text"] = df["text"].astype(str)
        return dict(zip(df["slide"], df["text"]))
    except FileNotFoundError:
        print(f"找不到腳本檔案: {script_path}")
        return {}

# 7.定義影片之聲音功能
- text:代表輸入是一段文字
_ 如果文字裡中文，則不使用英文voice，改用中文voice

In [56]:
def detect_voice_and_locale(text):
    if re.search(r"[\u4e00-\u9fff]", text):
        return (VOICE_ID_ZH, "zh-TW")
    return (VOICE_ID_EN, "en-US")

# 8.定義上傳圖片之功能
- 輸入型態:image/png
- 並要求上傳圖片之 asset id

In [57]:
def upload_image(img_path: Path):
    """上傳圖片到 HeyGen"""
    headers = {
        "x-api-key": HEYGEN_API_KEY,
        "Content-Type": "image/png",
    }
    with open(img_path, "rb") as f:
        file_data = f.read()

    r = requests.post(UPLOAD_URL_V1, headers=headers, data=file_data, params={"type": "image"})
    if r.status_code != 200:
        print(f"Upload failed: {r.text}")
        return None
    print(f"Uploaded: {img_path.name}")
    return r.json()["data"]["id"]

# 9.定義影片之功能
- slide number 單純是為了讓我知道哪個投影片製作時出問題(拿掉也不影響 code 之運行)
- header 那段:設定 HTTP Header (傳 API Key、告訴伺服器我要傳 json 檔的形式
- playload(影片的設定)
  - caption；字幕
  - video_input:
    - scale:頭像大小
    - offset:頭像位置
    - fit:頭像如何填滿畫面
  - background:
    - fit:contain (保持畫面塞滿不裁切)
  - dimension:調整影片大小
  - "test": False (非測試模式)

In [58]:
def create_video(asset_id, text, slide_number):
    voice_id, locale = detect_voice_and_locale(text)
    headers = {
        "x-api-key": HEYGEN_API_KEY,
        "Content-Type": "application/json"
    }

    payload = {
        "caption": True,
        "video_inputs": [
            {
                "character": {
                    "type": "talking_photo",
                    "talking_photo_id": TALKING_PHOTO_ID,
                    "scale": 0.3,
                    "offset": {"x": 0.4, "y": 0.4},
                    "talking_style": "stable",
                    "fit": "cover"
                },
                "voice": {
                    "type": "text",
                    "voice_id": voice_id,
                    "input_text": text,
                    "speed": 1.0,
                    "locale": locale
                },
                "background": {
                    "type": "image",
                    "image_asset_id": asset_id,
                    "fit": "contain"
                }
            }
        ],
        "dimension": {
            "width": 1280,
            "height": 720
        },
        "test": False
    }

    try:
        r = requests.post(GENERATE_URL_V2, headers=headers, json=payload)
        if r.status_code == 200:
            data = r.json()
            vid = data.get("data", {}).get("video_id")
            if vid:
                print(f"Job Submitted! ID: {vid}")
                return vid
        print(f"Job failed: Status {r.status_code}. Response: {r.text[:100]}...")
    except Exception as e:
        print(f"Network Error: {e}")

    return None

# 10.主程式流程(整合的概念)
- 步驟:載入腳本圖片，上傳圖片，建立影片任務給heygen
- 許多的print 都是為了讓我知道弄到哪了，出現問題時，方便我解決問題
- time.sleep(1):避免請求時間太短，因此設立
- 因為是免費版，無法直接獲得產生之影片的 video id ，因此需要去 heygen UI 介面直接下載影片

In [ ]:
def main():
    print("HeyGen小廢片影片生成器")

    scripts = load_scripts(SCRIPT_PATH)
    pngs = sorted(PNG_DIR.glob("*.png"))

    if not scripts or not pngs:
        print("缺少腳本或投影片圖片，請檢查 'scripts/script.xlsx' 和 'outputs/slides_png/'")
        return

    print(f"找到 {len(pngs)} 張投影片，準備發送 {len(scripts)} 個任務...")

    for i, img_path in enumerate(pngs, 1):
        text = scripts.get(i, "")
        if not text: continue

        print(f"\n--- Processing Slide {i} ---")

        asset_id = upload_image(img_path)
        if not asset_id: continue

        create_video(asset_id, text, i)

        time.sleep(1)
        
    print(" 請手動至 HeyGen UI 介面 (Projects) 查看進度並下載影片。")


main()

HeyGen影片產生
找到 3 張投影片，準備發送 3 個任務...

--- Processing Slide 1 ---
Uploaded: slide_01.png
Job Submitted! ID: 8e8d645528974dbe8742237142364715

--- Processing Slide 2 ---
Uploaded: slide_02.png
Job Submitted! ID: 5d80634629774893b065892f441b57d4

--- Processing Slide 3 ---
Uploaded: slide_03.png
Job Submitted! ID: 65509da248f94646a6762653af5d3b62
 請手動至 HeyGen UI 介面 (Projects) 查看進度並下載影片。


# 11.合併單頁式簡報(利用FFmeg)
- <記得務必下載影片，並將影片重新命名為 vedio1、vedio2 之形式，再來執行這段>
- 我自己是將影片放在 "C:\Users\User\Downloads" (如在不同地方，請自行更改)
- 會自動將編號小的影片放在前面，依序且合併


In [1]:
import glob
import subprocess
import re

def natural_sort_key(s):
    return [int(t) if t.isdigit() else t for t in re.findall(r'\d+|\D+', s)]

videos = glob.glob(r"C:\Users\User\Downloads\video*.mp4")
videos = sorted(videos, key=natural_sort_key)

if not videos:
    raise ValueError("找不到任何 video*.mp4")

with open("list.txt", "w", encoding="utf-8") as f:
    for v in videos:
        f.write(f"file '{v}'\n")

cmd = [
    "ffmpeg","-y",
    "-f", "concat",
    "-safe", "0",
    "-i", "list.txt",
    "-c", "copy",
    "output.mp4"
]

subprocess.run(cmd, check=True)


CompletedProcess(args=['ffmpeg', '-y', '-f', 'concat', '-safe', '0', '-i', 'list.txt', '-c', 'copy', 'output.mp4'], returncode=0)